In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PATH_TO_MEDS_CSV = # TODO path to manually cleaned treatment file
meds_df = pd.read_csv(PATH_TO_MEDS_CSV)
processed_datadir = # TODO path to processed data using other scripts
pd_totals_df = pd.read_csv(processed_datadir + 'PD_totals_across_time.csv')
pd_baseline_df = pd.read_csv(processed_datadir + 'PD_baseline.csv')
pd_other_df = pd.read_csv(processed_datadir + 'PD_other_across_time.csv')

In [ ]:
included_trts = {'LEVODOPA','RASAGILINE'}
meds_df = meds_df.loc[meds_df['CMTRT'].isin(included_trts)]
meds_df['STARTDT_date'] = pd.to_datetime(meds_df['STARTDT'])
meds_df.sort_values(by='STARTDT_date', inplace=True)
first_line_meds_df = meds_df.drop_duplicates(subset='PATNO', keep='first')

In [ ]:
pd_totals_df['NUPDRS3_max'] = pd_totals_df[['NUPDRS3_untreated','NUPDRS3_off',\
                                            'NUPDRS3_off','NUPDRS3_maob']].max(skipna=True, axis=1)
pd_totals_df['mds23'] = pd_totals_df[['NUPDRS2','NUPDRS3_max']].sum(axis=1)
df = pd_other_df[['PATNO','AGE']].rename(columns={'AGE': 'age'})
df = df.merge(pd_totals_df[['PATNO','mds23']], on='PATNO')
df = df.merge(pd_baseline_df[['PATNO','CNO']].rename(columns={'CNO':'site'}, inplace=True), on='PATNO', )
df = df.merge(first_line_meds_df[['PATNO','CMTRT']].rename(columns={'CMTRT':'treatment'}, inplace=True), on='PATNO')
df['treatment'] = np.where(df['treatment']=='LEVODOPA', 1, 0) # 1 for levodopa, 0 for rasagiline
del df['PATNO']
df.to_csv('line1_levodopa_rasagiline_age_mds23_disdur.csv',index=False)